# Projet maison n° 5

On s'intéresse au fichier sur la Couverture 2G, 3G, 4G en France par opérateur de juillet 2015 fourni par l'ARCEP : "couverture-2g-3g-4g-en-france-par-operateur-juillet-2015.csv".

[°] Ce fichier donne le taux de couverture par commune, par opérateur (Orange France, Bouygues Telecom, SFR, Free Mobile, Tout Opérateur), par type de couverture (Couverture population, Couverture surfacique) et par réseau (2G, 3G, 4G).

1. Chargez le fichier dans un DataFrame.
2. Transformez ce DataFrame large en un DataFrame long en isolant les colonnes d'identification d'une part et les colonnes de valeurs de couverture d'autre part.
3. Rajoutez 3 colonnes "Opérateur", "Type Couverture" et "Réseau" calculées à partir de la colonne "variable" obtenue en 2 (utilisez les valeurs indiquées entre parenthèses ci-dessus [°], ces valeurs sont utilisées dans les tests).
4. A partir de ce DataFrame, obtenez un objet de type Series dont les valeurs correspondent à la colonne "value" obtenue en 2 et dont le multi-index contient les autres colonnes du DataFrame sauf la colonne "variable" obtenue en 2. Puis remontez les index "Opérateur", "Type Couverture" et "Réseau" en colonnes multi-indexées. Vous devez obtenir un DataFrame avec 7 niveaux d'index et 3 niveaux de colonnes.
5. Chargez le DataFrame des communes déjà utilisé ("correspondance-code-insee-code-postal.csv") et effectuez une jointure sur le code INSEE avec le DataFrame obtenu en 3.
6. Pour le réseau 4G et la couverture surfacique, calculez la moyenne des couvertures par statut de commune et par opérateur. Pour quel couple (statut, opérateur) a-t-on la moyenne de la couverture la plus faible ?

**Nota bene** : les différentes instructions doivent être réalisées dans les fonctions prédéfinies pour passer les tests. Il est néanmoins possible de définir des fonctions annexes appelées par les fonctions prédéfinies.

In [82]:
# import
import pandas as pd
import os
import re 

os.chdir(r'C:\Users\benja\OneDrive\Documents\Telecom\Kit-data-science\Session5')

In [25]:
# 1) Chargez le fichier dans un DataFrame
def load_data():
    df = pd.read_csv(r'couverture-2g-3g-4g-en-france-par-operateur-juillet-2015.csv', sep = ';', header = 0)
    return df

In [73]:
# load_data
df = load_data()

In [74]:
# 2) Transformez ce DataFrame large en un DataFrame long
def data_wide2long():
    df = load_data()
    id_list = list(df.columns[0:6]) + [df.columns[-1]]
    df1 = pd.melt(frame = df, id_vars = id_list, value_vars = list(df.columns[6:-1]))
    return df1

In [137]:
# data_wide2long
df = data_wide2long()

In [138]:
# 3) Rajoutez 3 colonnes "Opérateur", "Type Couverture" et "Réseau"
def data_add3columns():
    df = data_wide2long()
    operateur = "Orange France|Bouygues Telecom|SFR|Free Mobile|Par au moins un opérateur"
    couverture = "Couverture population|Couverture surfacique"
    reseau = '2G|3G|4G'
    mapping = {'Par au moins un opérateur' : 'Tout Opérateur'}

    df["Opérateur"] = df["variable"].apply(lambda x : re.search(operateur, x)[0])
    df["Opérateur"] = df["Opérateur"].apply(lambda x : mapping.get(x, x))
    df["Type Couverture"] = df["variable"].apply(lambda x : re.search(couverture, x)[0])
    df["Réseau"] = df["variable"].apply(lambda x : re.search(reseau, x)[0])
    return df

In [229]:
# data_add3columns
df = data_add3columns()

In [292]:
# 4) Transformez ce DataFrame en multi-index
def data_multi_index():
    df = data_add3columns()
    multindex = list(df.columns)
    multindex.remove('variable')
    multindex.remove('value')
    df = df.pivot_table(values='value', index=multindex, columns = ["Opérateur", "Type Couverture", "Réseau"])
    df.unstack().unstack().unstack()
    return df

In [296]:
# data_multi_index
df = data_multi_index()

In [249]:
# 5) Effectuez une jointure sur le code INSEE
def data_join_geo():
    df = data_add3columns()
    geo =  pd.read_csv(r'C:\Users\benja\OneDrive\Documents\Telecom\Kit-data-science\Session2\correspondance-code-insee-code-postal.csv', sep = ';', header = 0)
    df = pd.merge(df, geo, left_on='CODE INSEE', right_on='Code INSEE')
    return df

In [252]:
# data_join_geo
df = data_join_geo()

In [274]:
# 6) Pour quels statut et opérateur a-t-on la couverture la plus faible ?
def data_get_worst_statut_operateur():
    df = data_join_geo()
    worst_statut_operateur = None
    df = df.loc[(df['Réseau'] == "4G") & (df["Type Couverture"] == "Couverture surfacique")]
    worst_statut_operateur = df.groupby(['Statut', 'Opérateur']).mean().value.idxmin()
    return worst_statut_operateur

In [275]:
# data_get_worst_statut_operateur
data_get_worst_statut_operateur()

('Commune simple', 'Free Mobile')

In [276]:
import unittest

class Lesson5Tests(unittest.TestCase):
    
    def test_01_load_data(self):
        df = load_data()
        # test shape
        self.assertEqual(df.shape, (36594, 37))
        
    def test_02_data_wide2long(self):
        df = data_wide2long()
        # test shape
        self.assertEqual(df.shape, (1097820, 9))
        
    def test_03_data_add3columns(self):
        df = data_add3columns()
        # test "Opérateur" values
        s1 = set(df['Opérateur'].unique())
        s2 = set(['Orange France', 'Bouygues Telecom', 'SFR', 'Free Mobile', 'Tout Opérateur'])
        self.assertSetEqual(s1, s2)
        # test "Type Couverture" values
        s1 = set(df['Type Couverture'].unique())
        s2 = set(['Couverture population', 'Couverture surfacique'])
        self.assertSetEqual(s1, s2)
        # test "Réseau" values
        s1 = set(df['Réseau'].unique())
        s2 = set(['2G', '3G', '4G'])
        self.assertSetEqual(s1, s2)
        
    def test_04_data_multi_index(self):
        df = data_multi_index()
        # test shape
        self.assertEqual(df.shape, (36594, 30))
        # test multi-index de l'index
        self.assertEqual(len(df.index.levels), 7)
        # test multi-index des columns
        self.assertEqual(len(df.columns.levels), 3)
        
    def test_05_data_join_geo(self):
        df = data_join_geo()
        # test length
        self.assertEqual(len(df), 1097820)
        
    def test_06_data_get_worst_statut_operateur(self):
        statut_operateur = data_get_worst_statut_operateur()
        # test length
        self.assertEqual(statut_operateur, ('Commune simple', 'Free Mobile'))


In [277]:
# run tests
test_suite = unittest.makeSuite(Lesson5Tests)
runner = unittest.TextTestRunner(verbosity=2)
runner.run(test_suite)

test_01_load_data (__main__.Lesson5Tests) ... ok
test_02_data_wide2long (__main__.Lesson5Tests) ... ok
test_03_data_add3columns (__main__.Lesson5Tests) ... ok
test_04_data_multi_index (__main__.Lesson5Tests) ... FAIL
test_05_data_join_geo (__main__.Lesson5Tests) ... ok
test_06_data_get_worst_statut_operateur (__main__.Lesson5Tests) ... ok

FAIL: test_04_data_multi_index (__main__.Lesson5Tests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\benja\AppData\Local\Temp/ipykernel_22424/2874662635.py", line 33, in test_04_data_multi_index
    self.assertEqual(df.shape, (36594, 30))
AssertionError: Tuples differ: (295955, 0) != (36594, 30)

First differing element 0:
295955
36594

- (295955, 0)
+ (36594, 30)

----------------------------------------------------------------------
Ran 6 tests in 24.884s

FAILED (failures=1)


<unittest.runner.TextTestResult run=6 errors=0 failures=1>